# Applying A* and dijkstra algorithm on Single Shortest Path Problem (SSP) further comparing the efficiency

# The Technique (A* Algorithm)

It is a searching algorithm that is used to find the shortest path between an initial and a final point.

It is a handy algorithm that is often used for map traversal to find the shortest path to be taken. A* was initially designed as a graph traversal problem, to help build a robot that can find its own course. It still remains a widely popular algorithm for graph traversal.

It searches for shorter paths first, thus making it an optimal and complete algorithm. An optimal algorithm will find the least cost outcome for a problem, while a complete algorithm finds all the possible outcomes of a problem.

Another aspect that makes A* so powerful is the use of weighted graphs in its implementation. A weighted graph uses numbers to represent the cost of taking each path or course of action. This means that the algorithms can take the path with the least cost, and find the best route in terms of distance and time.

# The Problem (Single Shortest Path Problem)
The Single Shortest Path (SSP) problem consists of finding the shortest paths between a given vertex v and all other vertices in the graph. Algorithms such as Breadth-First-Search (BFS) for unweighted graphs.

# Procedure(A* Algorithm)

1.  Initialize the open list
2.  Initialize the closed list
    put the starting node on the open 
    list (you can leave its f at zero)

3.  while the open list is not empty
    a) find the node with the least f on 
       the open list, call it "q"

    b) pop q off the open list
  
    c) generate q's 8 successors and set their 
       parents to q
   
    d) for each successor
        i) if successor is the goal, stop search
        
        ii) else, compute both g and h for successor
          successor.g = q.g + distance between 
                              successor and q
          successor.h = distance from goal to 
          successor (This can be done using many 
          ways, we will discuss three heuristics- 
          Manhattan, Diagonal and Euclidean 
          Heuristics)
          
          successor.f = successor.g + successor.h

        iii) if a node with the same position as 
            successor is in the OPEN list which has a 
           lower f than successor, skip this successor

        iV) if a node with the same position as 
            successor  is in the CLOSED list which has
            a lower f than successor, skip this successor
            otherwise, add  the node to the open list
     end (for loop)
  
    e) push q on the closed list
    end (while loop)

# Code(A* Algorithm)

In [5]:
from collections import defaultdict
import copy
from queue import PriorityQueue


def serialize(state):
    result = []
    for row in state:
        for col in row:
            result.append(str(col))
    return ':'.join(result)


def deserialize(serialized):
    splitted = serialized.split(':')
    splitted = [int(x) for x in splitted]
    return [splitted[:3], splitted[3:6], splitted[6:]]


def get_neighbours(state):
    deserialized = deserialize(state)
    neighbours = []
    blank_i = -1
    blank_j = -1

    for i in range(0, 3):
        for j in range(0, 3):
            if deserialized[i][j] == 0:
                blank_i, blank_j = i, j
                break

    i = blank_i
    j = blank_j

    if i > 0:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i - 1][j]
        new_matrix[i - 1][j] = 0
        neighbours.append(serialize(new_matrix))
    if i < 2:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i + 1][j]
        new_matrix[i + 1][j] = 0
        neighbours.append(serialize(new_matrix))
    if j > 0:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i][j - 1]
        new_matrix[i][j - 1] = 0
        neighbours.append(serialize(new_matrix))
    if j < 2:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i][j + 1]
        new_matrix[i][j + 1] = 0
        neighbours.append(serialize(new_matrix))

    return zip(neighbours, [1 for x in neighbours])


def h(state):
    deserialized = deserialize(state)
    H = 0
    for i in range(0, 3):
        for j in range(0, 3):
            H += abs(deserialized[i][j] % 3 - j) + abs(deserialized[i][j] / 3 - i)
    return H


def in_open_set_with_lowest_heuristic_guess(open_set, heuristic_guess):
    result, min_guess = None, float('inf')
    for v in open_set:
        if v in heuristic_guess:
            guess = heuristic_guess[v]
            if guess < min_guess:
                result = v
                min_guess = guess
    return result


def astar_lloyd(start_node, target_node, h):
    start_node = serialize(start_node)
    target_node = serialize(target_node)

    open_set = set([start_node])

    parents = {}
    parents[start_node] = None

    cheapest_paths = defaultdict(lambda: float('inf'))
    cheapest_paths[start_node] = 0

    heuristic_guess = defaultdict(lambda: float('inf'))
    heuristic_guess[start_node] = h(start_node)

    path_found = False
    iteration = 0
    while len(open_set) > 0:
        # O(1)
        current_node = in_open_set_with_lowest_heuristic_guess(open_set, heuristic_guess)
        if current_node == target_node:
            path_found = True
            break

        open_set.remove(current_node)
        for (neighbour_node, weight) in get_neighbours(current_node):
            new_cheapest_path = cheapest_paths[current_node] + weight
            if new_cheapest_path < cheapest_paths[neighbour_node]:
                parents[neighbour_node] = current_node
                cheapest_paths[neighbour_node] = new_cheapest_path
                heuristic_guess[neighbour_node] = new_cheapest_path + h(neighbour_node)
                if neighbour_node not in open_set:
                    open_set.add(neighbour_node)
        iteration += 1

    path = []
    if path_found:
        while target_node is not None:
            path.append(target_node)
            target_node = parents[target_node]
        path.reverse()
    return (path, iteration)

# Result(A* Algorithm)

In [6]:
if __name__ == '__main__':
    start_node = [[2, 3, 5], [1, 4, 0], [7, 8, 6]]
    target_node = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    print('A* Path and Cost respectively are:')
    (path, iteration) = astar_lloyd(start_node, target_node, h)
    print('Path: \n', path)
    print('Cost:', iteration)

A* Path and Cost respectively are:
Path: 
 ['2:3:5:1:4:0:7:8:6', '2:3:5:1:4:6:7:8:0', '2:3:5:1:4:6:7:0:8', '2:3:5:1:0:6:7:4:8', '2:0:5:1:3:6:7:4:8', '0:2:5:1:3:6:7:4:8', '1:2:5:0:3:6:7:4:8', '1:2:5:3:0:6:7:4:8', '1:2:5:3:6:0:7:4:8', '1:2:0:3:6:5:7:4:8', '1:0:2:3:6:5:7:4:8', '0:1:2:3:6:5:7:4:8', '3:1:2:0:6:5:7:4:8', '3:1:2:6:0:5:7:4:8', '3:1:2:6:4:5:7:0:8', '3:1:2:6:4:5:0:7:8', '3:1:2:0:4:5:6:7:8', '0:1:2:3:4:5:6:7:8']
Cost: 199


# Procedure(Dijkstra Algorithm)

Below are the detailed steps used in Dijkstra’s algorithm to find the shortest path from a single source vertex to all other vertices in the given graph. 
Algorithm


1. Create a set sptSet (shortest path tree set) that keeps track of vertices included in shortest path tree, i.e., whose minimum distance from source is calculated and finalized. Initially, this set is empty. 
2. Assign a distance value to all vertices in the input graph. Initialize all distance values as INFINITE. Assign distance value as 0 for the source vertex so that it is picked first. 

3. While sptSet doesn’t include all vertices:


*   Pick a vertex u which is not there in sptSet and has minimum distance value.
*   Include u to sptSet.

*   Update distance value of all adjacent vertices of u. To update the distance values, iterate through all adjacent vertices. For every adjacent vertex v, if the sum of a distance value of u (from source) and weight of edge u-v, is less than the distance value of v, then update the distance value of v.








# Code(Dijkstra Algorithm)

In [7]:
from collections import defaultdict
import copy
from queue import PriorityQueue


def serialize(state):
    result = []
    for row in state:
        for col in row:
            result.append(str(col))
    return ':'.join(result)


def deserialize(serialized):
    splitted = serialized.split(':')
    splitted = [int(x) for x in splitted]
    return [splitted[:3], splitted[3:6], splitted[6:]]


def get_neighbours(state):
    deserialized = deserialize(state)
    neighbours = []
    blank_i = -1
    blank_j = -1

    for i in range(0, 3):
        for j in range(0, 3):
            if deserialized[i][j] == 0:
                blank_i, blank_j = i, j
                break

    i = blank_i
    j = blank_j

    if i > 0:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i - 1][j]
        new_matrix[i - 1][j] = 0
        neighbours.append(serialize(new_matrix))
    if i < 2:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i + 1][j]
        new_matrix[i + 1][j] = 0
        neighbours.append(serialize(new_matrix))
    if j > 0:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i][j - 1]
        new_matrix[i][j - 1] = 0
        neighbours.append(serialize(new_matrix))
    if j < 2:
        new_matrix = copy.deepcopy(deserialized)
        new_matrix[i][j] = new_matrix[i][j + 1]
        new_matrix[i][j + 1] = 0
        neighbours.append(serialize(new_matrix))

    return zip(neighbours, [1 for x in neighbours])


def dijkstra(start_node, target_node):
    start_node = serialize(start_node)
    target_node = serialize(target_node)

    visited = set()
    D = defaultdict(lambda: float('inf'))
    D[start_node] = 0

    pq = PriorityQueue()
    pq.put((0, start_node))

    parent = dict()
    parent[start_node] = None
    path_found = False
    iteratrion = 0
    while not pq.empty():
        (dist, current_node) = pq.get()
        if current_node == target_node:
            path_found = True
            break
        visited.add(current_node)

        for (neighbour, distance_from_current_node) in get_neighbours(current_node):
            if neighbour not in visited:
                old_cost = D[neighbour]
                new_cost = D[current_node] + distance_from_current_node
                if new_cost < old_cost:
                    pq.put((new_cost, neighbour))
                    D[neighbour] = new_cost
                    parent[neighbour] = current_node
        iteratrion += 1

    path = []
    if path_found:
        path.append(target_node)
        while True:
            parent_node = parent[target_node]
            if parent_node is None:
                break
            path.append(parent_node)
            target_node = parent_node
        path.reverse()
    return (path, iteratrion)

# Result(Dijkstra Algorithm)

In [8]:
if __name__ == '__main__':
    start_state = [[2, 3, 5], [1, 4, 0], [7, 8, 6]]
    target_state = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]

    print('Dijkstra Path and Cost respectively are:')
    (path, iteration) = dijkstra(start_state, target_state)
    print('Path: \n', path)
    print('Cost:', iteration)

Dijkstra Path and Cost respectively are:
Path: 
 ['2:3:5:1:4:0:7:8:6', '2:3:5:1:4:6:7:8:0', '2:3:5:1:4:6:7:0:8', '2:3:5:1:0:6:7:4:8', '2:0:5:1:3:6:7:4:8', '0:2:5:1:3:6:7:4:8', '1:2:5:0:3:6:7:4:8', '1:2:5:3:0:6:7:4:8', '1:2:5:3:6:0:7:4:8', '1:2:0:3:6:5:7:4:8', '1:0:2:3:6:5:7:4:8', '0:1:2:3:6:5:7:4:8', '3:1:2:0:6:5:7:4:8', '3:1:2:6:0:5:7:4:8', '3:1:2:6:4:5:7:0:8', '3:1:2:6:4:5:0:7:8', '3:1:2:0:4:5:6:7:8', '0:1:2:3:4:5:6:7:8']
Cost: 12649


# Remarks

Even though Dijkstra's algorithm and the A* algorithm both find the same shortest paths, the A* algorithm does it almost 63.5 times faster! While Dijkstra's algorithm produced the output after 12649 iterations, it only took 199(depending on iteration) for the A* algotithm.

However, it should be noted that the efficiency of the A* algorithm is highly dependent on its evaluation function, and with the wrong function, the results could be even worse than Dijkstra.

At the end given that we have a good heuristic guess on our problem, it is definitely more efficient to use the A* algorithm compared to Dijkstra's algorithm, although this won't always be the case as it can be highly dependent on the problem at hand.